In [16]:
import pandas as pd
from sqlalchemy import create_engine

from config import api_key
import requests
import json
import time
import datetime as dt

# Store CSV into DataFrame

In [2]:
# Read company info into dataframe

csv_file = "Resources/securities.csv"
company = pd.read_csv(csv_file)
company.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [3]:
# Read company financials into dataframe

csv_file = "Resources/fundamentals.csv"
company_financial = pd.read_csv(csv_file)
company_financial.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [4]:
# Read company historical prices into dataframe
csv_file = "Resources/prices.csv"
company_prices = pd.read_csv(csv_file)
company_prices.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


# Clean DateFrames

# Filter companies for health care sector and generate list of tickers

In [5]:
# Filter companies in health care sector
company_hc = company.loc[company["GICS Sector"] == "Health Care"].reset_index(drop=True)

final_hc_company = company_hc[["Ticker symbol","Security","GICS Sector"]].copy()

final_hc_company.head()

,Ticker symbol,Security,GICS Sector
0,ABT,Abbott Laboratories,Health Care
1,ABBV,AbbVie,Health Care
2,AET,Aetna Inc,Health Care
3,A,Agilent Technologies Inc,Health Care
4,ALXN,Alexion Pharmaceuticals,Health Care


In [6]:
company_ticker = final_hc_company[["Ticker symbol"]].copy()
company_ticker.head()

,Ticker symbol
0,ABT
1,ABBV
2,AET
3,A
4,ALXN


# Extract lastest financial information for each company and inner join with tickers list to obtain financial info for all health care companies

In [7]:
# Extract columns interested: Ticker Symbol, Period Ending, Net Cash Flow, Total Assets,
#Total Equity, Total Liabilities, Total Revenue, Earnings Per Share
financial_extracted = company_financial[["Ticker Symbol","Period Ending",
                                         "Net Cash Flow","Total Assets",
                                         "Total Equity","Total Liabilities",
                                         "Total Revenue","Earnings Per Share"]].copy()


financial_extracted = financial_extracted.rename(columns = {"Ticker Symbol":"Ticker symbol"})

total_rows = len(financial_extracted.index)

In [8]:
# Extract the latest financial information for each company
financial_extracted.iloc[0,:]

# Loop through dataframe to obtain latest financials for each company
i=1
latest_financial = []
for i in range(1,total_rows):
    current_record = financial_extracted.iloc[i,:]
    last_record = financial_extracted.iloc[i-1,:]
    
    if current_record["Ticker symbol"] != last_record["Ticker symbol"]:
        latest_financial.append(last_record)

# Append last record to the list
latest_financial.append(financial_extracted.iloc[total_rows-1,:])

In [9]:
# convert to dataframe
latest_financial_df = pd.DataFrame(latest_financial).reset_index(drop = True)

In [10]:
# Inner join financials table with tickers to obtain financials for all Health Care companies
financials_final = pd.merge(company_ticker, latest_financial_df, on = "Ticker symbol", how = "inner")

financials_final.head()

,Ticker symbol,Period Ending,Net Cash Flow,Total Assets,Total Equity,Total Liabilities,Total Revenue,Earnings Per Share
0,ABT,2015-12-31,938000000.0,4.124700e+10,2.121100e+10,2.003600e+10,2.040500e+10,2.94
1,ABBV,2015-12-31,51000000.0,5.305000e+10,3.945000e+09,4.910500e+10,2.285900e+10,3.15
2,ALXN,2016-12-31,-44000000.0,1.325300e+10,8.694000e+09,4.559000e+09,3.084000e+09,NaN
3,ABC,2016-09-30,574390000.0,3.365620e+10,2.129404e+09,3.152680e+10,1.468497e+11,6.73
4,AMGN,2016-12-31,-903000000.0,7.762600e+10,2.987500e+10,4.775100e+10,2.299100e+10,NaN


# Request 2019 stock prices through API and calculate average close price for each company

In [11]:
ticker_list = company_ticker["Ticker symbol"].to_list()

ticker_symbol_list = [ ]
date_list = [ ]
close = [ ]

for ticker in ticker_list:
    base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='
    url = f'{base_url}{ticker}&apikey={api_key}'
    response = requests.get(url).json()
    
    try:
        ticker_symbol = response['Meta Data']['2. Symbol']
        response_dict = response['Monthly Adjusted Time Series']
    
    except KeyError:
        print(response)
        continue
    
    for date in response_dict.keys():
        if '2019' in date:
            ticker_symbol_list.append(ticker_symbol)
            date_list.append(date)
            close.append(response_dict[date]['4. close'])
    time.sleep(12)

In [14]:
# Convert equry results to dataframe
df_2019 = pd.DataFrame({ "Ticker symbol":ticker_symbol_list, 
                        "Date":date_list, 
                        "Close Price":close})
df_2019["Close Price"] = pd.to_numeric(df_2019["Close Price"])
df_2019.head()

,Ticker symbol,Date,Close Price
0,ABT,2019-12-31,86.86
1,ABT,2019-11-29,85.45
2,ABT,2019-10-31,83.61
3,ABT,2019-09-30,83.67
4,ABT,2019-08-30,85.32


In [42]:
#calculate average by company - avg_prices_2019
avg_prices_2019 = df_2019.groupby(['Ticker symbol']).mean()

avg_prices_2019 = avg_prices_2019.rename(columns={"Close Price":"Avg_prices_2019"})
avg_prices_2019.head()

,Avg_prices_2019
Ticker symbol,
A,76.255833
ABBV,77.736667
ABC,82.844167
ABT,81.861667
AGN,158.764167


# Clean prices data and calculate 2016 average stock price per company

In [17]:
company_prices.rename(columns={'symbol': 'Ticker symbol'}, inplace=True)
company_prices.head()

,date,Ticker symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [25]:
company_prices_2 = company_prices.loc[(company_prices['date'] >= '2016-01-01 00:00:00') & 
                                      (company_prices['date'] <= '2016-12-31 00:00:00')][["Ticker symbol","close"]]
company_prices_2.head()

,Ticker symbol,close
0,WLTW,125.839996
1,WLTW,119.980003
2,WLTW,114.949997
3,WLTW,116.620003
4,WLTW,114.970001


In [45]:
avg_prices_2016 = company_prices_2.groupby(['Ticker symbol']).mean()
avg_prices_2016 = avg_prices_2016.rename(columns={"close":"Avg_prices_2016"})
avg_prices_2016.head()

,Avg_prices_2016
Ticker symbol,
A,43.369960
AAL,38.183849
AAP,155.488135
AAPL,104.604008
ABBV,60.667976


# Connect to local database

In [29]:
rds_connection_string = "postgres:admin@localhost:5432/NYSE_HC_Companies"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [30]:
engine.table_names()

['avg_prices_2016', 'avg_prices_2019', 'final_hc_company', 'financials_final']

# Use pandas to load DataFrame into database

In [17]:
final_hc_company.to_sql(name='final_hc_company', con=engine, if_exists='replace', index=False)

In [18]:
financials_final.to_sql(name='financials_final', con=engine, if_exists='replace', index=False)

In [46]:
avg_prices_2016.to_sql(name='avg_prices_2016', con=engine, if_exists='replace')

In [47]:
avg_prices_2019.to_sql(name='avg_prices_2019', con=engine, if_exists='replace')

# Confirm data has been added by querying the tables

In [33]:
pd.read_sql_query('select * from final_hc_company', con=engine).head()

,Ticker symbol,Security,GICS Sector
0,ABT,Abbott Laboratories,Health Care
1,ABBV,AbbVie,Health Care
2,AET,Aetna Inc,Health Care
3,A,Agilent Technologies Inc,Health Care
4,ALXN,Alexion Pharmaceuticals,Health Care


In [34]:
pd.read_sql_query('select * from financials_final', con=engine).head()

,Ticker symbol,Period Ending,Net Cash Flow,Total Assets,Total Equity,Total Liabilities,Total Revenue,Earnings Per Share
0,ABT,2015-12-31,938000000.0,4.124700e+10,2.121100e+10,2.003600e+10,2.040500e+10,2.94
1,ABBV,2015-12-31,51000000.0,5.305000e+10,3.945000e+09,4.910500e+10,2.285900e+10,3.15
2,ALXN,2016-12-31,-44000000.0,1.325300e+10,8.694000e+09,4.559000e+09,3.084000e+09,NaN
3,ABC,2016-09-30,574390000.0,3.365620e+10,2.129404e+09,3.152680e+10,1.468497e+11,6.73
4,AMGN,2016-12-31,-903000000.0,7.762600e+10,2.987500e+10,4.775100e+10,2.299100e+10,NaN


In [48]:
pd.read_sql_query('select * from avg_prices_2016', con=engine).head()

,Ticker symbol,Avg_prices_2016
0,A,43.369960
1,AAL,38.183849
2,AAP,155.488135
3,AAPL,104.604008
4,ABBV,60.667976


In [49]:
pd.read_sql_query('select * from avg_prices_2019', con=engine).head()

,Ticker symbol,Avg_prices_2019
0,A,76.255833
1,ABBV,77.736667
2,ABC,82.844167
3,ABT,81.861667
4,AGN,158.764167
